In [6]:
%pip install flask-cors


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # ייבוא של CORS
import base64
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests

# Load environment variables from .env file
load_dotenv()

app = Flask(__name__)
CORS(app)  # הוספת CORS ליישום שלך

API_KEY=os.getenv("API_KEY")
BASE_URL=os.getenv("BASE_URL")
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# Function to encode the image
def image_url_to_base64(image_url):

    print(f"Downloading image from URL: {image_url}")
    response = requests.get(image_url)
    if response.status_code != 200:
        raise ValueError("Failed to download the image")
    return base64.b64encode(response.content).decode('utf-8')


@app.route('/color_instructions', methods=['POST'])
def color_instructions():
    try:
        data = request.json
        image_path = data.get('image_path')
        print(f"Received image path: {image_path}")
        if not image_path:
            return jsonify({"error": "No image path provided"}), 400

        # Getting the base64 string
        base64_image = image_url_to_base64(image_path)
        print(f"Base64 image: {base64_image}...")  # Print the first 30 characters of the base64 string

        response = client.chat.completions.create(
            model="gemini-2.0-flash",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": ".תן לי הוראות מפורטות איך לצבוע את התמונה הנתונה, תן פרוט על כל אלמנט בתמונה באיזה צבע לצבוע אותו ",  
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
        )

        return jsonify(response.choices[0].message.content)
    except Exception as e:
        return jsonify({"error": str(e)}), 500
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.2:5000
Press CTRL+C to quit
127.0.0.1 - - [19/May/2025 21:33:07] "OPTIONS /color_instructions HTTP/1.1" 200 -


Received image path: https://res.cloudinary.com/magicdraw/image/upload/v1743025701/moi-raskraski-lev-14-gigapixel-art-scale-2_00x %281%29.jpg.png


127.0.0.1 - - [19/May/2025 21:33:08] "POST /color_instructions HTTP/1.1" 500 -
